In [63]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import os, os.path
import pathlib
from pathlib import Path

import pandas as pd
import re

In [144]:
IMG_SIZE = 450
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [32]:
# example of loading the vgg16 model
from tensorflow.keras.applications.vgg16 import VGG16
# load model
model = VGG16(include_top=False, classes=2, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [33]:
model.trainable = False
model._layers.pop()
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 450, 450, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 450, 450, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 450, 450, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 225, 225, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 225, 225, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 225, 225, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 112, 112, 128)     0     

In [137]:
cwd = os.getcwd()
data_dir = pathlib.Path(cwd)
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))
for f in list_ds.take(5):
    print(f.numpy())


b'/home/freebreadstix/Documents/projects/LA-Hacks-2020/lahacks_2020/normal/300.jpg'
b'/home/freebreadstix/Documents/projects/LA-Hacks-2020/lahacks_2020/normal/340.jpg'
b'/home/freebreadstix/Documents/projects/LA-Hacks-2020/lahacks_2020/normal/40.jpg'
b'/home/freebreadstix/Documents/projects/LA-Hacks-2020/lahacks_2020/potholes/296.jpg'
b'/home/freebreadstix/Documents/projects/LA-Hacks-2020/lahacks_2020/normal/94.jpg'


In [138]:
def get_label(file_path):
  # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
    return parts[-2] == CLASS_NAMES

In [141]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
    return tf.image.resize(img, [IMG_SIZE, IMG_SIZE])

In [142]:
def process_path(file_path):
    label = get_label(file_path)
  # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [145]:
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

NameError: in user code:

    <ipython-input-142-3752848d5f37>:2 process_path  *
        label = get_label(file_path)
    <ipython-input-138-965c69ee9e99>:5 get_label  *
        return parts[-2] == CLASS_NAMES

    NameError: name 'CLASS_NAMES' is not defined


In [140]:
get_label(list_ds.take(1))

ValueError: Attempt to convert a value (<TakeDataset shapes: (), types: tf.string>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.TakeDataset'>) to a Tensor.

In [44]:
cwd = os.getcwd()
data_folder = pathlib.Path(cwd)
data_folder

PosixPath('/home/freebreadstix/Documents/projects/LA-Hacks-2020/lahacks_2020')

In [25]:
len(model.trainable_variables)

0

In [55]:
normal_list = list(data_folder.glob('normal/*.jpg'))
len(normal_list)
pothole_list = list(data_folder.glob('potholes/*.jpg'))
len(pothole_list)

329

In [62]:
combined = []
for image_path in normal_list:
    combined.append((image_path, 0))
for image_path in  pothole_list:
    combined.append((image_path, 1))
    
shuffled = shuffle(combined)
train, test = train_test_split(shuffled, test_size=0.2)

In [133]:
Image.open(train[0][0]).size
np.asarray(Image.open(train[0][0]), dtype=np.float32)*1./255

array([[[0.70980394, 0.6745098 , 0.60784316],
        [0.7882353 , 0.7529412 , 0.6862745 ],
        [0.7176471 , 0.67058825, 0.60784316],
        ...,
        [0.6039216 , 0.5686275 , 0.53333336],
        [0.6784314 , 0.6431373 , 0.6156863 ],
        [0.4509804 , 0.41568628, 0.3882353 ]],

       [[0.7019608 , 0.6666667 , 0.6       ],
        [0.78039217, 0.74509805, 0.6784314 ],
        [0.70980394, 0.6627451 , 0.6       ],
        ...,
        [0.5764706 , 0.5411765 , 0.5058824 ],
        [0.68235296, 0.64705884, 0.61960787],
        [0.4627451 , 0.42745098, 0.4       ]],

       [[0.68235296, 0.64705884, 0.5803922 ],
        [0.77254903, 0.7372549 , 0.67058825],
        [0.70980394, 0.6627451 , 0.6       ],
        ...,
        [0.54509807, 0.50980395, 0.4745098 ],
        [0.6509804 , 0.6156863 , 0.5882353 ],
        [0.44705883, 0.4117647 , 0.38431373]],

       ...,

       [[0.33333334, 0.3019608 , 0.25882354],
        [0.40392157, 0.37254903, 0.32941177],
        [0.36078432, 0

In [134]:
def format_example(tup):
    image_path, label = tup
    image = Image.open(image_path.as_posix())
    height, width = np.array(image.size, np.float32)
    channels = np.asarray(image)  * 1./255
    image_tensor = [height, width, channels]
    image_tensor = tf.cast(image_tensor, tf.float32)
    image = tf.image.resize_with_pad(image_tensor, IMG_SIZE, IMG_SIZE, antialias=False)
    return image, label

In [135]:
format_example(train[0])

ValueError: Can't convert Python sequence with mixed types to Tensor.

In [ ]:
tf.image.resize_with_pad(
    image, IMG_SIZE, IMG_SIZE, method=ResizeMethod.BILINEAR,
    antialias=False
)

In [ ]:

model2 = tf.Sequential([
    model,
    
])